In [81]:
import json
from collections import defaultdict

# Function to convert time format to the desired schema
def convert_time_to_schema(time_str):
    # Check if the time is in standard "HH:MM AM/PM" format
    if isinstance(time_str, str) and ' ' in time_str:
        time, clock = time_str.split(' ')
        return {"time": time, "clock": clock, "notification": True}
    # If it's a numerical value (e.g., 0.6666666667), try to handle it
    elif isinstance(time_str, (int, float)):
        # You can add custom handling for numbers here if needed
        # For example, converting 0.6666666667 to a standard time, or just defaulting to None.
        return {"time": str(time_str), "clock": "", "notification": False}
    else:
        # In case of unexpected format, return a default value or skip it
        return {"time": "", "clock": "", "notification": False}

# Function to process the input data and generate the new schema
def process_prayer_times(input_file, output_file):
    # Read the input data from the JSON file
    with open(input_file, 'r') as infile:
        data = json.load(infile)

    # Prepare the output structure
    output_data = defaultdict(dict)

    # Iterate over the months and dates
    for month, entries in data.items():
        for entry in entries:
            date_str = entry["Date"]
            day = int(date_str.split()[0])  # Extract the day number from "01 November 2024"
            date = str(day)

            # Build the structure for each prayer time
            output_data[month][date] = {
                "Fajr": {
                    "Adhaan": convert_time_to_schema(entry["Fajr Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Fajr Iqamah"]),
                },
                "Dhuhr": {
                    "Adhaan": convert_time_to_schema(entry["Dhuhr Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Dhuhr Iqamah"]),
                },
                "Asr": {
                    "Adhaan": convert_time_to_schema(entry["Asr Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Asr Iqamah"]),
                },
                "Maghrib": {
                    "Adhaan": convert_time_to_schema(entry["Maghrib Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Maghrib Iqamah"]),
                },
                "Isha": {
                    "Adhaan": convert_time_to_schema(entry["Isha Adhaan"]),
                    "Iqamah": convert_time_to_schema(entry["Isha Iqamah"]),
                }
            }

    # Write the transformed data into the output JSON file
    with open(output_file, 'w') as outfile:
        json.dump(output_data, outfile, indent=4)

# Example usage
input_file = 'prayer_time.json'  # Input JSON file
output_file = 'prayer_time_transformed.json'  # Output JSON file

process_prayer_times(input_file, output_file)


In [62]:
import json

# Function to split time and clock safely
def split_time_clock(time_str):
    try:
        time_parts = time_str.split(' ')
        if len(time_parts) == 2:
            return time_parts[0], time_parts[1]
        else:
            # Handle cases where the format might not be as expected
            return time_str, ""
    except Exception as e:
        print(f"Error processing time string '{time_str}': {e}")
        return "", ""

# Load the prayer_time.json and prayerTime.json files
with open('prayer_time.json', 'r') as file1:
    prayer_time_data = json.load(file1)

with open('prayerTime.json', 'r') as file2:
    prayer_time_json_data = json.load(file2)

# Loop through each month in the prayer_time_data and copy Sunrise time to prayerTime.json
for month, days in prayer_time_data.items():
    for day_info in days:
        # Extract the Date from the prayer_time.json format
        date_parts = day_info['Date'].split(' ')
        day = date_parts[0]
        month_name = date_parts[1]
        year = date_parts[2]

        # Ensure the date and month exist in the prayerTime.json structure
        if year not in prayer_time_json_data:
            prayer_time_json_data[year] = {}

        if month_name not in prayer_time_json_data[year]:
            prayer_time_json_data[year][month_name] = {}

        # Copy Sunrise time and add to the prayerTime.json structure
        sunrise_time = day_info['Sunrise']
        sunrise_parts = sunrise_time.split(' ')

        # Prepare the Sunrise entry in the format required in prayerTime.json
        prayer_time_json_data[year][month_name][str(int(day))] = {
            "Sunrise": {
                "time": sunrise_parts[0],
                "clock": sunrise_parts[1] if len(sunrise_parts) > 1 else "",
                "notification": True
            }
        }

        # Also add the other prayer timings (this is part of your expected structure)
        prayer_time_json_data[year][month_name][str(int(day))].update({
            "Fajr": {
                "Adhaan": {
                    "time": split_time_clock(day_info["Fajr Adhaan"])[0],
                    "clock": split_time_clock(day_info["Fajr Adhaan"])[1],
                    "notification": True
                },
                "Iqamah": {
                    "time": split_time_clock(day_info["Fajr Iqamah"])[0],
                    "clock": split_time_clock(day_info["Fajr Iqamah"])[1],
                    "notification": True
                }
            },
            "Dhuhr": {
                "Adhaan": {
                    "time": split_time_clock(day_info["Dhuhr Adhaan"])[0],
                    "clock": split_time_clock(day_info["Dhuhr Adhaan"])[1],
                    "notification": True
                },
                "Iqamah": {
                    "time": split_time_clock(day_info["Dhuhr Iqamah"])[0],
                    "clock": split_time_clock(day_info["Dhuhr Iqamah"])[1],
                    "notification": True
                }
            },
            "Asr": {
                "Adhaan": {
                    "time": split_time_clock(day_info["Asr Adhaan"])[0],
                    "clock": split_time_clock(day_info["Asr Adhaan"])[1],
                    "notification": True
                },
                "Iqamah": {
                    "time": split_time_clock(day_info["Asr Iqamah"])[0],
                    "clock": split_time_clock(day_info["Asr Iqamah"])[1],
                    "notification": True
                }
            },
            "Maghrib": {
                "Adhaan": {
                    "time": split_time_clock(day_info["Maghrib Adhaan"])[0],
                    "clock": split_time_clock(day_info["Maghrib Adhaan"])[1],
                    "notification": True
                },
                "Iqamah": {
                    "time": split_time_clock(day_info["Maghrib Iqamah"])[0],
                    "clock": split_time_clock(day_info["Maghrib Iqamah"])[1],
                    "notification": True
                }
            },
            "Isha": {
                "Adhaan": {
                    "time": split_time_clock(day_info["Isha Adhaan"])[0],
                    "clock": split_time_clock(day_info["Isha Adhaan"])[1],
                    "notification": True
                },
                "Iqamah": {
                    "time": split_time_clock(day_info["Isha Iqamah"])[0],
                    "clock": split_time_clock(day_info["Isha Iqamah"])[1],
                    "notification": True
                }
            }
        })

# Save the updated prayerTime.json
with open('updated_prayerTime.json', 'w') as file:
    json.dump(prayer_time_json_data, file, indent=4)

print("Data has been updated and saved to 'updated_prayerTime.json'")


Data has been updated and saved to 'updated_prayerTime.json'
